# Вступительное задание

## ШКОЛА ПО БАЙЕСОВСКИМ МЕТОДАМ В ГЛУБИННОМ ОБУЧЕНИИ

Летняя школа по современным методам глубинного обучения и байесовскому подходу

26-31 августа 2017, Москва

## Потапов Анатолий 4115 ФБМФ МФТИ

Минимизируем функцию:

![formula](images/formula.png)

In [1]:
import numpy as np
from scipy import special
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.datasets import make_classification

Используйте scipy.special для вычисления численно неустойчивых функций
https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special

In [3]:
def lossf(w, X, y, l1, l2):
    """
    Вычисление функции потерь.

    :param w: numpy.array размера  (M,) dtype = np.float
    :param X: numpy.array размера  (N, M), dtype = np.float
    :param y: numpy.array размера  (N,), dtype = np.int
    :param l1: float, l1 коэффициент регуляризатора 
    :param l2: float, l2 коэффициент регуляризатора 
    :return: float, value of loss function
    """
    def stable_log1exp(x):
        big = (x > 35.0)*1.0
        big = big * x
        small = (x < 35.0)*1.0
        small = small * x
        small = np.log(1 + np.exp(small))
        return big+small
    
    # Вам необходимо вычислить значение функции потерь тут, решение может занимать 1 строку
    lossf = np.sum(stable_log1exp(-y*np.sum(X*w, axis=1))) + l1*np.sum(np.abs(w)) + l2*np.sum(w**2) 
    return lossf

In [4]:
def gradf(w, X, y, l1, l2):
    """
    Вычисление градиента функции потерь.

    :param w: numpy.array размера  (M,), dtype = np.float
    :param X: numpy.array размера  (N, M), dtype = np.float
    :param y: numpy.array размера  (N,), dtype = np.int
    :param l1: float, l1 коэффициент регуляризатора 
    :param l2: float, l2 коэффициент регуляризатора 
    :return: numpy.array размера  (M,), dtype = np.float, gradient vector d lossf / dw
    """
    
    # Вам необходимо вычислить градиент функции потерь тут, решение может занимать 1 строку
        
    gradw = np.sum(
        special.expit(-y*np.sum(X*w, axis=1))[:, np.newaxis] * (-y[:, np.newaxis]*X),
        axis = 0
    ) + 2*l2*np.abs(w) + l1*np.sign(w)
    
    return gradw

In [5]:
class LR(ClassifierMixin, BaseEstimator):
    
    def __init__(self, lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0):
        self.l1 = l1
        self.l2 = l2
        self.w = None
        self.lr = lr
        self.verbose = verbose
        self.num_iter = num_iter

    def fit(self, X, y):
        """
        Обучение логистической регрессии.
        Настраивает self.w коэффициенты модели.

        Если self.verbose == True, то выводите значение 
        функции потерь на итерациях метода оптимизации. 

        :param X: numpy.array размера  (N, M), dtype = np.float
        :param y: numpy.array размера  (N,), dtype = np.int
        :return: self
        """
        n, d = X.shape
 
        # Задайте начальное приближение вектора весов
        w = np.random.uniform(low=-1.0, high=1.0, size=d)
        
        # Настройте параметры функции потерь с помощью градиентного спуска   
        for iter in range(self.num_iter):
            grad = gradf(w, X, y, self.l1, self.l2)
            w = np.add(w, -self.lr*grad)
            # Вывод функции потерь
            if self.verbose == True and iter % 30 == 0:
                print "Значение функции потерь на итерации {0}: {1}".format(iter, lossf(w, X, y, self.l1, self.l2))
               
        self.w = w
        return self
    
    def predict_proba(self, X):
        """
        Предсказание вероятности принадлежности объекта к классу 1.
        Возвращает np.array размера (N,) чисел в отрезке от 0 до 1.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :return: numpy.array размера  (N,), dtype = np.int
        """
        # Вычислите вероятности принадлежности каждого 
        # объекта из X к положительному классу, используйте
        # эту функцию для реализации LR.predict
        
        probs = special.expit(np.sum(X*self.w, axis=1))
        return probs
    
    def predict(self, X):
        """
        Предсказание класса для объекта.
        Возвращает np.array размера (N,) элементов 1 или -1.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :return:  numpy.array размера  (N,), dtype = np.int
        """

        # Вычислите предсказания для каждого объекта из X
        predicts = np.sign(np.sum(X*self.w, axis=1))
        return predicts 

In [6]:
def test_work():
    print "Start test"
    X, y = make_classification(n_features=100, n_samples=1000)
    y = 2 * (y - 0.5)

    try:
        clf = LR(lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0)
    except Exception:
        assert False, "Создание модели завершается с ошибкой"
        return

    try:
        clf = clf.fit(X, y)
    except Exception:
        assert False, "Обучение модели завершается с ошибкой"
        return

    assert isinstance(lossf(clf.w, X, y, 1e-3, 1e-3), float), "Функция потерь должна быть скалярной и иметь тип np.float"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).shape == (100,), "Размерность градиента должна совпадать с числом параметров"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).dtype == np.float, "Вектор градиента, должен состоять из элементов типа np.float"
    assert clf.predict(X).shape == (1000,), "Размер вектора предсказаний, должен совпадать с количеством объектов"
    assert np.min(clf.predict_proba(X)) >= 0, "Вероятности должны быть не меньше, чем 0"
    assert np.max(clf.predict_proba(X)) <= 1, "Вероятности должны быть не больше, чем 1"
    assert len(set(clf.predict(X))) == 2, "Метод предсказывает больше чем 2 класса на двух классовой задаче"
    print "End tests"
def test_work():
    print "Start test"
    X, y = make_classification(n_features=100, n_samples=1000)
    y = 2 * (y - 0.5)

    try:
        clf = LR(lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0)
    except Exception:
        assert False, "Создание модели завершается с ошибкой"
        return

    try:
        clf = clf.fit(X, y)
    except Exception:
        assert False, "Обучение модели завершается с ошибкой"
        return

    assert isinstance(lossf(clf.w, X, y, 1e-3, 1e-3), float), "Функция потерь должна быть скалярной и иметь тип np.float"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).shape == (100,), "Размерность градиента должна совпадать с числом параметров"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).dtype == np.float, "Вектор градиента, должен состоять из элементов типа np.float"
    assert clf.predict(X).shape == (1000,), "Размер вектора предсказаний, должен совпадать с количеством объектов"
    assert np.min(clf.predict_proba(X)) >= 0, "Вероятности должны быть не меньше, чем 0"
    assert np.max(clf.predict_proba(X)) <= 1, "Вероятности должны быть не больше, чем 1"
    assert len(set(clf.predict(X))) == 2, "Метод предсказывает больше чем 2 класса на двух классовой задаче"
    print "End tests"


In [7]:
X, y = make_classification(n_features=100, n_samples=1000)
y = 2 * (y - 0.5)
clf = LR(lr= 1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=True)
clf = clf.fit(X, y)
clf.w

Значение функции потерь на итерации 0: 35893.1886814
Значение функции потерь на итерации 30: 5013.91289013
Значение функции потерь на итерации 60: 3750.69311214
Значение функции потерь на итерации 90: 5503.63775143
Значение функции потерь на итерации 120: 4722.56329836
Значение функции потерь на итерации 150: 4608.45094853
Значение функции потерь на итерации 180: 4966.84668066
Значение функции потерь на итерации 210: 5137.20498881
Значение функции потерь на итерации 240: 4984.76734963
Значение функции потерь на итерации 270: 3639.77447642
Значение функции потерь на итерации 300: 4752.3372167
Значение функции потерь на итерации 330: 4223.96410167
Значение функции потерь на итерации 360: 4740.21330235
Значение функции потерь на итерации 390: 4024.0010114
Значение функции потерь на итерации 420: 4125.11380716
Значение функции потерь на итерации 450: 4705.56271358
Значение функции потерь на итерации 480: 5415.69780755
Значение функции потерь на итерации 510: 4728.99424243
Значение функции 

array([  -0.54918831,  -14.89467622,   27.88663558,   -7.88867535,
        139.44070388,    6.73912665,  -19.48138449,    0.31787488,
        -16.39913365,  -26.20424781,    7.36188869,  -15.58978978,
         -7.94986454,  -10.11484377,  112.28848107,   -2.09364613,
        -30.42715269,   12.70918451,   13.9134559 ,    9.36732763,
        -13.59677232,  -18.39172443,  -19.94171865,    6.87963925,
         -0.34838264,   -2.13734373,  -20.64818967,   -1.30783227,
         -4.90367623,  -31.4108649 ,   -3.09932777,  -42.56933656,
          7.01175395,    2.2927014 ,   13.19433416,  -21.25098747,
          4.44785808,  -50.16201286,    7.08526424,    9.49000819,
         -7.64863948,  -11.67983641,   19.01551455,   -6.21290313,
        -17.1238397 ,   -3.69029826,   -9.30525685,   -8.23028356,
         -8.96479667,    8.32324949,   10.51243472,   13.18674271,
          9.7638033 ,  200.79443807,   -7.28313823,   29.72267359,
        -24.42873802,   32.00525683,  -28.49309366,   12.97418

In [8]:
test_work()

Start test
End tests


## Test how our model works

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
X, y = make_classification(n_features=100, n_samples=1000)
y = 2 * (y - 0.5)

### Train and test dataset

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Sklearn native logistic regression

In [12]:
scikit_lr = LogisticRegression()

In [13]:
%%time
scikit_lr.fit(X_train, y_train)

CPU times: user 32 ms, sys: 4 ms, total: 36 ms
Wall time: 34 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
print "Scikit model accuracy: {0}".format(accuracy_score(y_test, scikit_lr.predict(X_test)))

Scikit model accuracy: 0.885


### My logistic regression

In [15]:
my_lr = LR(lr=1, l1=0, l2=0, num_iter=1000, verbose=0)

In [16]:
%%time
my_lr = my_lr.fit(X, y)

CPU times: user 764 ms, sys: 412 ms, total: 1.18 s
Wall time: 1.18 s


In [17]:
print "My model accuracy: {0}".format(accuracy_score(y_test, my_lr.predict(X_test)))

My model accuracy: 0.905
